In [3]:
import re
import csv  
import time
import requests
import numpy as np
import pandas as pd
from csv import writer
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [4]:
#Get the soup object from specified url
def getSoup(url):
    try:
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        #print(url)
        return soup
    except Exception as e:
        print(e)
        pass

In [5]:
#Gets name from soup object
def getGameName(soup):
    try:
        if((soup.find(class_ = 'apphub_AppName').get_text()) == None):
            return
        else:
            gameName = soup.find(class_ = 'apphub_AppName').get_text()
            #print(gameName)
            return gameName
    except Exception as e:
        #print(e)
        pass

In [6]:
#Gets review from soup object
def getReview(soup):
    try:
        review = soup.find(class_ = 'game_review_summary positive').get_text()
        #print(review)
        return review
    except:
        pass

In [7]:
#Gets date from soup object
def getDate(soup):
    try:
        date = soup.find(class_ = 'date').get_text()
        #print(date)
        return date
    except:
        pass

In [8]:
#Gets developer name from soup object
def developerName(soup):
    try:
        developer = soup.find(href="https://store.steampowered.com/developer/doublefine?snr=1_5_9__2000").get_text()
        return developer
    except:
        pass

In [9]:
#Gets tags from soup object
def getTags(soup):
    try:
        tags = soup.find_all(class_ = 'app_tag')
        tagArray=[]
        cleanedTag = ''
        for i in tags:
            i.findNext()
            cleanedTag = i.get_text()
            cleanedTag = cleanedTag.replace('\t','')
            cleanedTag = cleanedTag.replace('\n','')
            cleanedTag = cleanedTag.replace('\r','')
            tagArray.append(cleanedTag)
        tagArray.pop()#Scrapper takes the + so we just remove it
        #print(tagArray)
        return(tagArray)
    except:
        pass

In [10]:
#Finding the different href
def getHref(url):
    try:
        soup = getSoup(url)
        links_with_text = [a['href'] for a in soup.find_all('a', href=True) if a.text]
        #print(links_with_text)
        return links_with_text
    except:
        pass

In [11]:
#This gets all information from a specific page. I dont end up using it.
def getPageInformation(url):
    try:
        soup = getSoup(url)
        try:
            gameName = getGameName(soup)
        except:
            print("no name")
        try:
            getReview(soup)
        except:
            print("no reviews")
        try:
            getDate(soup)
        except:
            print("no date")
        try:
            developerName(soup)
        except:
            print("no developer name")
        try:
            getTags(soup)
        except:
            print("no tags")
            
    except Exception as e:
        print(e)

In [12]:
'''#Getting all the game links from a specific steam page
url = 'https://store.steampowered.com/search/?term=dota+2'
PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)
driver.get(url)
hrefs = getHref(url)    
#print(hrefs)
f=open('test_file.txt','w')
for ele in hrefs:
    f.write(ele+'\n')
f.close()
driver.quit()'''

"#Getting all the game links from a specific steam page\nurl = 'https://store.steampowered.com/search/?term=dota+2'\nPATH = 'C:\\Program Files (x86)\\chromedriver.exe'\ndriver = webdriver.Chrome(PATH)\ndriver.get(url)\nhrefs = getHref(url)    \n#print(hrefs)\nf=open('test_file.txt','w')\nfor ele in hrefs:\n    f.write(ele+'\n')\nf.close()\ndriver.quit()"

In [13]:
#Got this function from: https://thispointer.com/python-how-to-append-a-new-row-to-an-existing-csv-file/
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [14]:
#Steam requires user to be over 18 to access some page. This is just a way to go around it
def ageCheck(driver):
    #If steam redirects you the will redirect you to a page which first 40 char are string1
    string1 = 'https://store.steampowered.com/agecheck/'
    #We check if it requires a age check
    if(driver.current_url[0:40] == string1):
        drp=Select(driver.find_element_by_id("ageYear"))
        drp.select_by_visible_text('1995')
        try:
            button = dri|ver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]')
        except:
            button = driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]')
        button.click()
    elif(driver.current_url == '#'):
        pass

In [16]:
dataBase = pd.read_csv('steam_href_user_review4.csv', encoding='latin1')
dataBase

,Href
0,https://store.steampowered.com/app/1089980/The...
1,https://store.steampowered.com/app/1229490/ULT...
2,https://store.steampowered.com/app/1144400/Sen...
3,https://store.steampowered.com/app/1055540/A_S...
4,https://store.steampowered.com/app/431730/Asep...
...,...
17544,https://store.steampowered.com/app/1444750/Dre...
17545,https://store.steampowered.com/app/16866/Civil...
17546,https://store.steampowered.com/app/621922/Trai...
17547,https://store.steampowered.com/app/434901/Car_...


In [21]:
csvFile = 'final_Data2.csv'
#Creates a csv file that will contain all the page info
with open('final_Data2.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Name', 'Review', 'Date', 'Dev name', 'Tags'])
df3 = pd.read_csv('final_Data2.csv', encoding='latin1')
df3

,Name,Review,Date,Dev name,Tags


In [22]:
def scrapePage(dataBase):
    for i in range(len(dataBase)):
        try:
            print('in try')
            soup = getSoup(dataBase['Href'][i])
            url = dataBase['Href'][i]
            PATH = 'C:\Program Files (x86)\chromedriver.exe'
            driver = webdriver.Chrome(PATH)
            driver.get(url)
            ageCheck(driver)
            print('after age')
            name = getGameName(soup)
            review = getReview(soup)
            date = getDate(soup)
            developer = developerName(soup)
            tags = getTags(soup)
            print(tags)
            row_content = [name, review, date, developer, tags]
            append_list_as_row('final_Data2.csv', row_content)
            driver.close()
        except:
            pass
            driver.close()

In [23]:
dataBase

,Href
0,https://store.steampowered.com/app/1089980/The...
1,https://store.steampowered.com/app/1229490/ULT...
2,https://store.steampowered.com/app/1144400/Sen...
3,https://store.steampowered.com/app/1055540/A_S...
4,https://store.steampowered.com/app/431730/Asep...
...,...
17544,https://store.steampowered.com/app/1444750/Dre...
17545,https://store.steampowered.com/app/16866/Civil...
17546,https://store.steampowered.com/app/621922/Trai...
17547,https://store.steampowered.com/app/434901/Car_...


In [ ]:
scrapePage(dataBase)

in try
after age
['Funny', 'Choose Your Own Adventure', 'Multiple Endings', 'Memes', 'Classic', 'Comedy', 'Singleplayer', 'Choices Matter', 'Point & Click', '2D', 'Family Friendly', 'Hand-drawn', 'Epic', 'Adventure', 'Indie', 'Old School', 'Action', 'Parody ', 'Interactive Fiction', 'Casual']
in try
after age
['Early Access', 'FPS', 'Spectacle fighter', 'Character Action Game', 'Arena Shooter', 'Blood', 'Singleplayer', 'Shooter', 'Retro', 'First-Person', 'Difficult', 'Robots', 'Silent Protagonist', 'Cyberpunk', 'Old School', '3D', 'Action', 'Sci-fi', 'Colorful', 'Stylized']
in try
after age
['Casual', 'Adventure', 'Visual Novel', 'Anime', 'Dating Sim', 'Romance', 'Cute', 'Story Rich', 'Singleplayer', 'NSFW', 'Multiple Endings', 'Great Soundtrack', 'World War II', 'Shooter']
in try
after age
['Adventure', 'Exploration', 'Indie', 'Cute', 'Relaxing', 'Pixel Graphics', '3D Platformer', 'Nature', 'Casual', 'Walking Simulator', 'Singleplayer', 'Great Soundtrack', 'Funny', 'Short', 'Female Pr

after age
None
in try
after age
['Puzzle', 'Indie', 'Difficult', 'Singleplayer', 'Pixel Graphics', 'Cute', 'Logic', '2D', 'Programming', 'Sokoban', 'Great Soundtrack', 'Casual', 'Psychological Horror', 'Minimalist', 'Atmospheric', 'Top-Down']
in try
after age
['Indie', 'Horror', 'Puzzle', 'Atmospheric', 'Mystery', '2D', 'Drama', 'Singleplayer', 'Hand-drawn', 'Psychological Horror', 'Old School', 'Retro', 'Thriller', 'Supernatural', 'RPG', 'Visual Novel', 'Puzzle Platformer', 'Narration', 'Conversation', 'Text-Based']
in try
after age
['FPS', 'Retro', 'Fast-Paced', 'Horror', 'Shooter', 'Lovecraftian', 'First-Person', 'Old School', "1990's", 'Dark Humor', 'Great Soundtrack', 'Action', 'Classic', 'Singleplayer', 'Level Editor', 'Atmospheric', 'Arena Shooter', 'PvP', 'Bullet Time', 'Blood']
in try
after age
['Movie', 'Short', 'Atmospheric', 'Story Rich', 'Great Soundtrack', 'Free to Play']
in try
after age
['Rhythm', 'Great Soundtrack', 'Music', 'Bullet Hell', 'Difficult', 'Indie', '2D', '

after age
['Casual', 'Adventure', 'Visual Novel', 'Anime', 'Story Rich', 'Romance', 'Dating Sim', 'Sports']
in try
after age
['Adventure', 'Puzzle', 'Mystery', 'Lovecraftian', 'Story Rich', 'Horror', 'Point & Click', 'Hidden Object', 'Exploration', 'Singleplayer', 'First-Person', '3D', 'Atmospheric']
in try
after age
['Choose Your Own Adventure', 'Interactive Fiction', 'Diplomacy', 'Visual Novel', 'Management', 'Medieval', 'RPG', 'Strategy', 'Text-Based', '2D', 'Story Rich', 'Choices Matter', 'Multiple Endings', 'Conversation', 'Fantasy', 'Dark Fantasy', 'Simulation', 'Gothic', 'Atmospheric', 'Adventure']
in try
after age
['Simulation', 'Singleplayer', 'Realistic', 'Funny', 'First-Person', 'Sandbox', 'Physics', 'Family Friendly', 'Adventure', 'Free to Play', 'Casual', 'Indie', 'Management', 'Immersive Sim', 'Life Sim', '3D', 'Story Rich', 'Relaxing', 'Resource Management', 'Replay Value']
in try
after age
['Anime', 'Visual Novel', 'Casual', 'Adventure', 'Simulation', 'Robots', 'Drama',

in try
after age
['Cute', 'Exploration', 'Adventure', 'Singleplayer', 'First-Person', 'Colorful', 'Open World', 'Funny', 'Sandbox', 'Simulation', 'Casual', 'Fantasy', 'Management', 'Indie', 'Great Soundtrack', 'Family Friendly', 'Early Access', 'Female Protagonist', 'Action', 'FPS']
in try
in try
after age
['Comedy', 'Co-op', 'Funny', 'Platformer', 'Multiplayer', 'Local Co-Op', 'Adventure', 'Indie', '2D', 'Action', 'Great Soundtrack', 'Online Co-Op', 'Cats', 'Casual', 'Puzzle', 'Level Editor', 'Local Multiplayer', 'Singleplayer', 'Difficult', 'Team-Based']
in try
after age
['Stealth', 'First-Person', 'Action', 'Steampunk', 'Assassin', 'Singleplayer', 'Atmospheric', 'Story Rich', 'Adventure', 'Multiple Endings', 'Dark', 'Magic', 'Dystopian ', 'FPS', 'RPG', 'Replay Value', 'Fantasy', 'Open World', 'Immersive Sim', 'Shooter']
in try
after age
['Base Building', 'Automation', 'Crafting', 'Open World', 'Multiplayer', 'Co-op', 'Resource Management', 'Building', 'Sandbox', 'Exploration', 'Adve

after age
None
in try
after age
['Visual Novel', 'Detective', 'Story Rich', 'Great Soundtrack', 'Anime', 'Mystery', 'Adventure', 'Singleplayer', 'Puzzle', 'Crime', 'Funny', 'Point & Click', 'Classic', 'Cult Classic', 'Comedy', 'Conversation', '2D', 'Memes', 'Casual', 'Dynamic Narration']
in try
after age
['Strategy', 'Medieval', 'Colony Sim', 'RTS', 'Multiplayer', 'City Builder', 'Classic', 'Simulation', 'Tactical', 'Historical', 'Economy', 'Singleplayer', 'Building', 'Sandbox', 'Adventure', 'Co-op', '2D', 'Action', 'Great Soundtrack', 'Remake']
in try
after age
['Anime', 'Metroidvania', 'Exploration', 'Female Protagonist', 'Bullet Hell', 'Pixel Graphics', 'Indie', 'Action-Adventure', 'Side Scroller', 'Action', 'Singleplayer', 'Platformer', 'Adventure', 'Cute', 'Difficult', 'Crowdfunded', '2D', 'Colorful', 'Fantasy', 'Magic']
in try
after age
['Puzzle', 'Hand-drawn', 'Indie', 'Adventure', 'Singleplayer', 'Beautiful', 'Casual', 'Atmospheric', 'Stylized', 'Short', '2D', 'Story Rich', 'My

...

after age
['Adventure', 'Indie', 'Point & Click', 'Mythology', 'Dark', 'Atmospheric', 'Singleplayer', 'Puzzle', '2D']
in try
after age
['Indie', 'Casual', 'Platformer', 'Short', 'Singleplayer', '2D', 'Action', 'Minimalist', 'Experimental', 'Great Soundtrack', 'Puzzle Platformer', 'Atmospheric', 'Space', 'Soundtrack', 'Music', 'Family Friendly', 'Controller', 'Sci-fi', 'Retro', 'Steam Machine']
in try
after age
['Action', 'Fighting', '2D Fighter', '2D', 'Anime']
in try
after age
None
in try
after age
['Design & Illustration', 'RPG', 'Utilities', 'Software']
in try
after age
['Action', 'FPS', 'Shooter', 'Classic', 'First-Person', 'Horror', 'Singleplayer', "1990's"]
in try
after age
['Action', 'FPS', 'Story Rich', 'Shooter', 'Atmospheric', 'Singleplayer', 'First-Person', 'Steampunk', 'Sci-fi']
in try
after age
['Indie', 'Adventure', 'Survival', 'Story Rich', 'Third Person', 'Singleplayer', 'Historical', 'Relaxing', 'Colorful', 'Short', 'Nature', 'Exploration', 'Controller', 'Episodic', 'B

after age
['Indie', 'Platformer', 'Action', 'Dinosaurs', 'Pixel Graphics', 'Adventure', 'Racing', 'Retro', 'Fast-Paced', 'Space', 'Great Soundtrack', 'Difficult', 'Time Attack', '2D', 'Local Multiplayer', 'Local Co-Op', '4 Player Local', 'Cute']
in try
after age
['Tactical RPG', 'Turn-Based Tactics', 'Visual Novel', 'Tabletop', 'Turn-Based Strategy', 'Female Protagonist', 'RPG', 'Strategy', 'Funny', 'Isometric', 'Story Rich', 'Hand-drawn', 'Turn-Based', 'Tactical', 'Colorful', 'Fantasy', 'Mystery', 'Singleplayer', 'Adventure', 'Indie']
in try
after age
None
in try
in try
after age
['Action', 'RPG', 'Indie', 'Early Access', 'Female Protagonist', 'Anime', 'Hack and Slash', 'Action RPG', 'JRPG']
in try
after age
['Indie', 'VR', 'Historical', 'Free to Play', 'Education']
in try
after age
['Sexual Content', 'Nudity', 'Adventure', 'Visual Novel', 'Indie', 'Casual', 'Violent', 'Anime', 'Dating Sim', 'Psychological Horror', 'Singleplayer']
in try
after age
['Casual', 'Hidden Object', 'Relaxing

in try
after age
['Sports', 'Simulation', 'Indie', 'VR', 'Free to Play']
in try
after age
['Fantasy', 'Action', 'Adventure', 'Indie', 'Action-Adventure', 'Character Action Game', 'Exploration', 'Platformer', '2D Platformer', 'Hack and Slash', 'Precision Platformer', 'Side Scroller', '2D', 'Pixel Graphics', 'Cartoony', 'Cinematic', 'Colorful', 'Stylized', 'Assassin', 'Atmospheric']
in try
after age
['VR', 'Music', 'Audio Production', 'Rhythm', 'Simulation', 'Replay Value', 'Education', 'Utilities', 'Indie', 'Casual', 'Sandbox', 'Early Access', 'Singleplayer', 'Immersive Sim', 'Immersive', 'Realistic']
in try
after age
['Sexual Content', 'Nudity', 'Casual', 'Indie', 'Action', 'Simulation', 'Visual Novel', 'Anime', 'Cute', 'Singleplayer', 'FPS']
in try
after age
['Casual', 'Indie', 'Strategy', 'Puzzle', 'Relaxing']
in try
after age
['Indie', 'Casual', 'Sports', 'VR', 'Football', 'Action', 'Arcade', 'Funny', 'Colorful', 'Character Customization']
in try
after age
['Indie', 'Adventure', 'RP

after age
['Casual', 'Singleplayer', '2D', 'Fantasy', 'Turn-Based Tactics', 'Free to Play', 'Turn-Based Strategy', 'Indie', 'Card Battler']
in try
after age
['Action', 'Indie', 'Soundtrack']
in try
after age
['Gore', 'Indie', 'Simulation', 'Horror', 'Comedy', 'Dark Comedy', 'Physics', 'Dark', 'Multiplayer', 'Parody ']
in try
after age
['Casual', 'Indie', 'Female Protagonist', 'LGBTQ+', 'Anime', 'Visual Novel', 'Sexual Content', 'Nudity']
in try
after age
['Horror', 'Choose Your Own Adventure', 'CRPG', 'JRPG', 'Trading', 'Survival Horror', 'Casual', 'Visual Novel', 'Exploration', 'Interactive Fiction', 'Open World', 'Point & Click', 'Tabletop', 'RPG', '2D', 'Comic Book', 'Cute', 'Isometric', 'Top-Down', 'Choices Matter']
in try
after age
['Simulation', 'Racing', 'Singleplayer', 'Realistic', 'Education', 'Great Soundtrack', 'Relaxing', 'Atmospheric', 'First-Person', 'Inventory Management', 'Management']
in try
after age
['Choose Your Own Adventure', 'RPG', 'Adventure', 'Indie', 'Text-Bas

after age
None
in try
after age
['Simulation', 'Strategy', 'Singleplayer', 'Family Friendly', 'Supernatural', 'Resource Management', 'Management', 'Relaxing', 'Funny', 'Colorful', 'City Builder', 'Economy', 'Cute', 'Third Person', 'Great Soundtrack']
in try
after age
['RPG', 'Open World', 'Choices Matter', 'Tactical', 'Magic', 'Replay Value', 'Multiple Endings', 'Exploration', 'Fantasy', 'Strategy RPG', 'CRPG', 'Story Rich', 'Atmospheric', 'Isometric', 'Pixel Graphics', 'Retro', 'Adventure', 'Singleplayer', 'Tactical RPG', 'Indie']
in try
after age
['Action']
in try
after age
['Indie', 'Casual', 'Adventure', 'Puzzle', 'Pixel Graphics', 'Funny']
in try
after age
['Casual', 'Indie', 'Puzzle', '2D', 'Physics', 'Minimalist', 'Singleplayer', 'Short', 'Relaxing', 'Fast-Paced']
in try
after age
['Strategy', 'Music']
in try
after age
['Indie', 'Action', 'Twin Stick Shooter', 'Multiplayer', 'Fast-Paced', '4 Player Local', 'Local Multiplayer', 'Tanks', 'PvP', 'Shooter', 'Top-Down', 'Arena Shoote

In [ ]:
steamtest = pd.read_csv('final_Data.csv', encoding='latin1')

In [ ]:
steamtest